In [3]:
import tensorflow as tf
import numpy as np

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    

## Basic Setup - Use MNIST and RNN simple cell to classify images

In [4]:
reset_graph()

n_steps = 28 
n_inputs = 28 
n_neurons = 200 
n_outputs = 10
learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) 
y = tf.placeholder(tf.int32, [None])

# Basic RNN Cell
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons) 
# Create the RNN cell
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
# Attach the fully_connected layer for classification
logits = tf.layers.dense(states, n_outputs, activation=None) 
# Calculate SoftMax-Cross-Entropy
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits) 
# Calculate Loss - which is a simple MSE 
loss = tf.reduce_mean(xentropy)
# Attach the optimizer - Adam
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) 
# Perform Back-Prop
training_op = optimizer.minimize(loss)
# Collect the top-k outputs
correct = tf.nn.in_top_k(logits, y, 1)
# Calculate Accuracy 
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))   

# Initializer for variables
init = tf.global_variables_initializer()

## Download MNIST DataSet

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


### Train

In [6]:
n_epochs = 100 
batch_size = 150

with tf.Session() as sess: 
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size) 
            X_batch = X_batch.reshape((-1, n_steps, n_inputs)) 
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch}) 
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test}) 
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

(0, 'Train accuracy:', 0.92666668, 'Test accuracy:', 0.9374001)
(1, 'Train accuracy:', 0.96666664, 'Test accuracy:', 0.9514001)
(2, 'Train accuracy:', 0.96000004, 'Test accuracy:', 0.95300007)
(3, 'Train accuracy:', 0.98000002, 'Test accuracy:', 0.96060014)
(4, 'Train accuracy:', 0.95333332, 'Test accuracy:', 0.97030014)
(5, 'Train accuracy:', 0.98666668, 'Test accuracy:', 0.96920013)
(6, 'Train accuracy:', 0.99333334, 'Test accuracy:', 0.97400016)
(7, 'Train accuracy:', 0.96000004, 'Test accuracy:', 0.96470016)
(8, 'Train accuracy:', 0.9666667, 'Test accuracy:', 0.96700013)
(9, 'Train accuracy:', 0.98000002, 'Test accuracy:', 0.97560012)
(10, 'Train accuracy:', 0.99333334, 'Test accuracy:', 0.97590005)
(11, 'Train accuracy:', 0.97333336, 'Test accuracy:', 0.97060013)
(12, 'Train accuracy:', 1.0, 'Test accuracy:', 0.97350013)
(13, 'Train accuracy:', 0.98666668, 'Test accuracy:', 0.97600007)
(14, 'Train accuracy:', 0.97333336, 'Test accuracy:', 0.97020012)
(15, 'Train accuracy:', 0.9933